# Project 1, Part 3, Meal Related Queries



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [1]:
import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.3.1 How many meals were purchased for all of AGM?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item. 

The sum of quantity in the line_items table will tell you the total meals purchased.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return only 1 row into a Pandas dataframe and should look similar to this: 

||total_meals_purchased|
|---|---|
|0|8228284|

In [8]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select sum(quantity) as total_meals_purchased
from line_items
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,total_meals_purchased
0,8228284


In [13]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from line_items
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,store_id,sale_id,line_item_id,product_id,quantity
0,1,1,1,1,2
1,1,1,2,2,2
2,1,2,1,1,4
3,1,2,2,3,2
4,1,2,3,4,1
...,...,...,...,...,...
5417969,5,227721,2,2,1
5417970,5,227721,3,3,1
5417971,5,227721,4,5,1
5417972,5,227721,5,6,1


# 1.3.2 How many meals were purchased for all of AGM by meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item. 

For meal_name, use the description column in the products table.

Sort by meal_name in alphabetical order.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return 8 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||meal_name|total_meals_purchased|
|---|---|---|
|0|Brocolli Stir Fry|913984|
|...|...|...|
|7|Tilapia Piccata|687237|

In [12]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select p.description as meal_name,
       sum(line.quantity) as total_meals_purchased
from line_items as line
     join products as p
     on line.product_id = p.product_id
group by meal_name
order by meal_name
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,meal_name,total_meals_purchased
0,Brocolli Stir Fry,913984
1,Chicken Salad,228561
2,Curry Chicken,1368884
3,Eggplant Lasagna,1599058
4,Pistachio Salmon,1828778
5,Spinach Orzo,456769
6,Teriyaki Chicken,1145013
7,Tilapia Piccata,687237


# 1.3.3 How many meals were purchased by store and by meal?

For store_name use the store's city.

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

For meal_name, use the description column in the products table.

Sort by store_name in alphabetical order, then by meal_name in alphabetical order.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.



The query should return 40 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||store_name|meal_name|total_meals_purchased|
|---|---|---|---|
|0|Berkeley|Brocolli Stir Fry|232038|
|...|...|...|...|
|39|Seattle|Tilapia Piccata|153448|

In [17]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select s.city as store_name,
       p.description as meal_name,
       sum(line.quantity) as total_meals_purchased
from line_items as line
     join products as p
         on line.product_id = p.product_id
     join stores as s
         on line.store_id = s.store_id
group by store_name, meal_name
order by store_name, meal_name
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,store_name,meal_name,total_meals_purchased
0,Berkeley,Brocolli Stir Fry,232038
1,Berkeley,Chicken Salad,57719
2,Berkeley,Curry Chicken,346508
3,Berkeley,Eggplant Lasagna,405637
4,Berkeley,Pistachio Salmon,464274
5,Berkeley,Spinach Orzo,115469
6,Berkeley,Teriyaki Chicken,290858
7,Berkeley,Tilapia Piccata,174252
8,Dallas,Brocolli Stir Fry,179885
9,Dallas,Chicken Salad,44756


# 1.3.4 How many meals were purchased by month?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the month_number (1 = January) and the month from the sale_date.

Sort by month_number.


Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.



The query should return 12 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||month_number|month|total_meals_purchased|
|---|---|---|---|
|0|1|January  |650319|
|...|...|...|...|
|11|12|December |695035|

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """


select extract(month from sa.sale_date) as month_number,
       to_char(sa.sale_date, 'Month') as month,
       sum(line.quantity) as total_meals_purchased
from line_items as line
     join sales as sa
         on line.sale_id = sa.sale_id and line.store_id = sa.store_id
group by month_number, month
order by month_number
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,month_number,month,total_meals_purchased
0,1,January,650319
1,2,February,631190
2,3,March,731635
3,4,April,687607
4,5,May,664820
5,6,June,677009
6,7,July,666087
7,8,August,752484
8,9,September,631580
9,10,October,741259


In [19]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select *
from sales
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,store_id,sale_id,customer_id,sale_date,total_amount
0,1,1,710,2020-01-01,48
1,1,2,1477,2020-01-01,132
2,1,3,1820,2020-01-01,48
3,1,4,1272,2020-01-01,96
4,1,5,1986,2020-01-01,36
...,...,...,...,...,...
1537612,5,227717,29429,2020-12-31,60
1537613,5,227718,31048,2020-12-31,96
1537614,5,227719,30206,2020-12-31,60
1537615,5,227720,29097,2020-12-31,12


# 1.3.5 How many meals were purchased by month and meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the month_number (1 = January) and the month from the sale_date.

For meal_name, use the description column in the products table.

Sort by month_number, then by meal_name in alphabetical order.


Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


**Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.**


The query should return 96 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||month_number|month|meal_name|total_meals_purchased|
|---|---|---|---|---|
|0|1|January  |Brocolli Stir Fry|72161|
|...|...|...|...|...|
|95|12|December |Tilapia Piccata|58260|

In [21]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select extract(month from sa.sale_date) as month_number,
       to_char(sa.sale_date, 'Month') as month,
       p.description as meal_name,
       sum(line.quantity) as total_meals_purchased
from line_items as line
     join sales as sa
         on line.sale_id = sa.sale_id and line.store_id = sa.store_id
     join products as p
         on line.product_id = p.product_id
group by month_number, month, meal_name
order by month_number, meal_name
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,month_number,month,meal_name,total_meals_purchased
0,1,January,Brocolli Stir Fry,72161
1,1,January,Chicken Salad,18003
2,1,January,Curry Chicken,108134
3,1,January,Eggplant Lasagna,126605
4,1,January,Pistachio Salmon,144053
...,...,...,...,...
91,12,December,Eggplant Lasagna,134666
92,12,December,Pistachio Salmon,154242
93,12,December,Spinach Orzo,38297
94,12,December,Teriyaki Chicken,96896


# 1.3.6 How many meals were purchased by day of week and meal?

Each record in the line_items table has a quantity which is the number of meals purchased for that line item.

Derive the dow (0 = Sunday) and the day_of_week from the sale_date.

For meal_name, use the description column in the products table.

Sort by dow, then by meal_name in alphabetical order

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.


The query should return 56 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||dow|day_of_week|meal_name|total_meals_purchased|
|---|---|---|---|---|
|0|0|Sunday   |Brocolli Stir Fry|172250|
|...|...|...|...|...|
|55|6|Saturday |Tilapia Piccata|135327|

In [22]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select extract(dow from sa.sale_date) as dow,
       to_char(sa.sale_date, 'Day') as day_of_week,
       p.description as meal_name,
       sum(line.quantity) as total_meals_purchased
from line_items as line
     join sales as sa
         on line.sale_id = sa.sale_id and line.store_id = sa.store_id
     join products as p
         on line.product_id = p.product_id
group by dow, day_of_week, meal_name
order by dow, meal_name
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,dow,day_of_week,meal_name,total_meals_purchased
0,0,Sunday,Brocolli Stir Fry,172250
1,0,Sunday,Chicken Salad,43110
2,0,Sunday,Curry Chicken,257216
3,0,Sunday,Eggplant Lasagna,301726
4,0,Sunday,Pistachio Salmon,344315
5,0,Sunday,Spinach Orzo,85968
6,0,Sunday,Teriyaki Chicken,215741
7,0,Sunday,Tilapia Piccata,128763
8,1,Monday,Brocolli Stir Fry,121717
9,1,Monday,Chicken Salad,30790
